In [8]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
from ase.io import read as read_ase
import icet

# find files 

In [4]:
simulation_folder = '/nfshome/winkelmann/CathodeSimulationResults/Lichtenberg_250603/'

simulation_concentrations = glob.glob(simulation_folder + 'Va_*')

structure_files = {}
for conc in simulation_concentrations:
    concentration = 100 - float(conc.split('/')[-1].strip('Va_'))
    if concentration == 13:
        concentration = 12.5
    elif concentration == 63:
        concentration = 62.5
    elif concentration == 88:
        concentration = 87.5
    structure_files[str(concentration)] = glob.glob(conc + '/*/kmc_Cathode_Ovito.XDATCAR')

In [5]:
# read structures using ASE
structures = {}
for conc, files in structure_files.items():
    structures[conc] = []
    for run in files:
        sim = read_ase(run, index=':')
        structures[conc].extend(sim)

# translate structure to energy

In [20]:
cluster_expansion_file = glob.glob(simulation_folder + '/program_files/mixing_energy_anchorTS_elasticnet.ce')
cluster_expansion = icet.ClusterExpansion.read(cluster_expansion_file[0])
dfs_energies = {}
for conc, struct_list in structures.items():
    energies = []
    timesteps = []
    for structure in struct_list:
        print(structure.info)
        print(structure)
        energies.append(cluster_expansion.predict(structure))
        #timesteps.append(structure.info['time_step'])
            
    dfs_energies[conc] = pd.DataFrame({'Energy (eV)': energies, 'Time Step': timesteps, 'Structures': struct_list})


{}
Atoms(symbols='Li432Ni576O1152X144', pbc=True, cell=[[68.229561, 0.0, 0.0], [0.0, 59.08848, 0.0], [0.0, -1.64135, 4.71521]])
{}
Atoms(symbols='Li432Ni576O1152X144', pbc=True, cell=[[68.229561, 0.0, 0.0], [0.0, 59.08848, 0.0], [0.0, -1.64135, 4.71521]])
{}
Atoms(symbols='Li432Ni576O1152X144', pbc=True, cell=[[68.229561, 0.0, 0.0], [0.0, 59.08848, 0.0], [0.0, -1.64135, 4.71521]])
{}
Atoms(symbols='Li432Ni576O1152X144', pbc=True, cell=[[68.229561, 0.0, 0.0], [0.0, 59.08848, 0.0], [0.0, -1.64135, 4.71521]])
{}
Atoms(symbols='Li432Ni576O1152X144', pbc=True, cell=[[68.229561, 0.0, 0.0], [0.0, 59.08848, 0.0], [0.0, -1.64135, 4.71521]])
{}
Atoms(symbols='Li432Ni576O1152X144', pbc=True, cell=[[68.229561, 0.0, 0.0], [0.0, 59.08848, 0.0], [0.0, -1.64135, 4.71521]])
{}
Atoms(symbols='Li432Ni576O1152X144', pbc=True, cell=[[68.229561, 0.0, 0.0], [0.0, 59.08848, 0.0], [0.0, -1.64135, 4.71521]])
{}
Atoms(symbols='Li432Ni576O1152X144', pbc=True, cell=[[68.229561, 0.0, 0.0], [0.0, 59.08848, 0.0], [0.

KeyboardInterrupt: 

In [ ]:
#save resulting dataframes to csv
for conc, df in dfs_energies.items():
    df.to_csv(f'{simulation_folder}energies_and_time_{conc}.csv', index=False)